In [7]:
import networkx as nx
import requests
import json
from pathlib import Path
from itertools import product

ENDPOINT = 'http://localhost:8000/api/'
ENDPOINT = 'http://domset.algorithm.engineering/api/'


def to_dimacs(G):
    n = G.number_of_nodes()
    m = G.number_of_edges()
    lines = []
    lines.append(f"p ds {n} {m}")
    for u, v in G.edges():
        lines.append(f"{u + 1} {v + 1}")
    return "\n".join(lines)

def add_tag(name, description = None, style=None):
    data = {
        'name': name,
    }

    if description is not None:
        data['description'] = description

    if style is not None:
        data['style'] = style

    response = requests.post(ENDPOINT + 'tags/new', json=data)
    return response.json()

In [8]:
add_tag('random', 'Random graph', 2)
add_tag('gnp', 'Random G(n,p) graph', 2)
add_tag('networkrepository.com', "Collected from http://networkrepository.com", 2)

add_tag('mesh', "Graph has an underlying mesh structure", 3)


{'status': 'success', 'tag_id': 11}

In [9]:
# gnp
for n in [10, 50, 100, 200, 500, 1000]:
    for avgdeg in [2, 5, 10, 15, 20, 50]:
        if avgdeg * 3 >= n:
            continue

        for _ in range(5):
            p = avgdeg / (n - 1)
            G = nx.gnp_random_graph(n, p)
            dimacs = to_dimacs(G)

            req = {
                "name": f"gnp",
                "description": f"Random G(n, p) graph with n={n} and p={p:.2E}",
                "data": dimacs,
                "submitted_by": "manpen",
                "tags": ["random", "gnp"],
            }
            
            url = ENDPOINT + 'instances/new'
            x = requests.post(url, json = req).json()
            #print(x)



In [10]:
# regular
for n in [10, 20, 50, 100, 200, 500]:
    for d in [2, 3, 4, 5, 10]:
        if 2*d > n: continue

        for seed in range(5):
            G = nx.random_regular_graph(d, n, seed = 1337 * seed + 1234)
            dimacs = to_dimacs(G)

            req = {
                "name": f"Random Regular",
                "description": f"Random Regular Graph {n} nodes and degree {d}",
                "data": dimacs,
                "submitted_by": "manpen",
                "tags": ["random", "regular"],
            }
            
            url = ENDPOINT + 'instances/new'
            x = requests.post(url, json = req).json()


In [11]:
# HyperCube
for d in range(3,16):
    G = nx.hypercube_graph(d)
    G = nx.convert_node_labels_to_integers(G)
    dimacs = to_dimacs(G)

    req = {
        "name": f"Hypercube d={d}",
        "description": f"Hypercube on {d} dimensions",
        "data": dimacs,
        "submitted_by": "manpen",
        "tags": ["regular", "mesh"],
    }
    
    url = ENDPOINT + 'instances/new'
    x = requests.post(url, json = req).json()

In [12]:
# Mesh
for d in [2, 3, 4]:
    sizes = [2, 3, 5, 10, 15]

    for dims in product( *[sizes for _ in range(d)] ):
        for periodic in [False, True]:
            dims = list(dims)
            if sorted(dims) != dims:
                continue

            G = nx.grid_graph(dims, periodic)
            if G.number_of_nodes() < 10:
                continue

            G = nx.convert_node_labels_to_integers(G)

            if d == 1:
                name = "Circle" if periodic else "Path"
            else:
                name = "Torus" if periodic else "Mesh"
                
            coords = "x".join(str(x) for x in dims) 
            print(coords)

            req = {
                "name": f"{name} {coords}",
                "description": f"{d}dim {name} {coords}",
                "data": to_dimacs(G),
                "submitted_by": "manpen",
                "tags": ["mesh"],
            }

            url = ENDPOINT + 'instances/new'
            x = requests.post(url, json = req).json()
            


2x5
2x5
2x10
2x10
2x15
2x15
3x5
3x5
3x10
3x10
3x15
3x15
5x5
5x5
5x10
5x10
5x15
5x15
10x10
10x10
10x15
10x15
15x15
15x15
2x2x3
2x2x3
2x2x5
2x2x5
2x2x10
2x2x10
2x2x15
2x2x15
2x3x3
2x3x3
2x3x5
2x3x5
2x3x10
2x3x10
2x3x15
2x3x15
2x5x5
2x5x5
2x5x10
2x5x10
2x5x15
2x5x15
2x10x10
2x10x10
2x10x15
2x10x15
2x15x15
2x15x15
3x3x3
3x3x3
3x3x5
3x3x5
3x3x10
3x3x10
3x3x15
3x3x15
3x5x5
3x5x5
3x5x10
3x5x10
3x5x15
3x5x15
3x10x10
3x10x10
3x10x15
3x10x15
3x15x15
3x15x15
5x5x5
5x5x5
5x5x10
5x5x10
5x5x15
5x5x15
5x10x10
5x10x10
5x10x15
5x10x15
5x15x15
5x15x15
10x10x10
10x10x10
10x10x15
10x10x15
10x15x15
10x15x15
15x15x15
15x15x15
2x2x2x2
2x2x2x2
2x2x2x3
2x2x2x3
2x2x2x5
2x2x2x5
2x2x2x10
2x2x2x10
2x2x2x15
2x2x2x15
2x2x3x3
2x2x3x3
2x2x3x5
2x2x3x5
2x2x3x10
2x2x3x10
2x2x3x15
2x2x3x15
2x2x5x5
2x2x5x5
2x2x5x10
2x2x5x10
2x2x5x15
2x2x5x15
2x2x10x10
2x2x10x10
2x2x10x15
2x2x10x15
2x2x15x15
2x2x15x15
2x3x3x3
2x3x3x3
2x3x3x5
2x3x3x5
2x3x3x10
2x3x3x10
2x3x3x15
2x3x3x15
2x3x5x5
2x3x5x5
2x3x5x10
2x3x5x10
2x3x5x15
2x3x5x15
2x3x

In [13]:
descs = {
    'pace19': 'Exact Instance of PACE 2019 / Vertex Cover',
    'pace20': 'Exact Instance of PACE 2020 / Treedepth',
    'pace21': 'Exact Instance of PACE 2020 / Cluster Editing',
    'pace22': 'Exact Instance of PACE 2020 / Directed Feedback Vertex Set',
    'pace23': 'Exact Instance of PACE 2020 / Twinwidth',
}



for x in Path("graphs").glob("**/*.gr"):
    if x.parent.name not in descs: 
        continue


    with open(x) as f:
        data = f.read()

    req = {
        "name": x.stem,
        "description": descs[x.parent.name],
        "data": data,
        "submitted_by": "manpen",
        "tags": [str(x.parent.name) + "exact"],
    }
    
    try:
        url = ENDPOINT + 'instances/new'
        x = requests.post(url, json = req).json()
        print(x)
    except:
        pass